## 类继承关系
```mermaid
classDiagram
    class Wrapping
    class BaseAccessor
    class BaseSRAccessor
    class BaseDFAccessor
    
    Wrapping <|-- BaseAccessor
    BaseAccessor <|-- BaseSRAccessor
    BaseAccessor <|-- BaseDFAccessor
```

## class BaseAccessor(Wrapping)

### 使用方式

### `__init__`
保存 `obj` 到 `self._obj`，然后使用 `wrapper/obj` 来调用 `Wrapping.__init__`。
```python
def __init__(self, obj: tp.SeriesFrame, wrapper: tp.Optional[ArrayWrapper] = None, **kwargs) -> None:
    checks.assert_instance_of(obj, (pd.Series, pd.DataFrame))

    self._obj = obj

    wrapper_arg_names = get_func_arg_names(ArrayWrapper.__init__) 
    grouper_arg_names = get_func_arg_names(ColumnGrouper.__init__)
    wrapping_kwargs = dict() 
    
    for k in list(kwargs.keys()):
        if k in wrapper_arg_names or k in grouper_arg_names:
            wrapping_kwargs[k] = kwargs.pop(k)  
    
    if wrapper is None:
        wrapper = ArrayWrapper.from_obj(obj, **wrapping_kwargs)
    else:
        wrapper = wrapper.replace(**wrapping_kwargs)
    
    Wrapping.__init__(self, wrapper, obj=obj, **kwargs)
```

### `__call__`
假如 `a` 是 `BaseAccessor` 实例，那么可以通过 `a(**kwargs)` 的方式调用 `Configured.replace` 方法来构建一个新的 `BaseAccessor` 实例。
```python
def __call__(self: BaseAccessorT, **kwargs) -> BaseAccessorT:
    return self.replace(**kwargs)
```

### indexing_func
根据 `pd_indexing_func` 创建新的实例：更新 `self._wrapper` 和 `self._obj`。
```python
def indexing_func(self: BaseAccessorT, pd_indexing_func: tp.PandasIndexingFunc, **kwargs) -> BaseAccessorT:
    new_wrapper, idx_idxs, _, col_idxs = self.wrapper.indexing_func_meta(pd_indexing_func, **kwargs)
    new_obj = new_wrapper.wrap(self.to_2d_array()[idx_idxs, :][:, col_idxs], group_by=False)
    
    if checks.is_series(new_obj):
        return self.replace(
            cls_=self.sr_accessor_cls,
            obj=new_obj,
            wrapper=new_wrapper
        )
    return self.replace(
        cls_=self.df_accessor_cls,
        obj=new_obj,
        wrapper=new_wrapper
    )
```

## class BaseSRAccessor(BaseAccessor)

```python
class BaseSRAccessor(BaseAccessor):
    def __init__(self, obj: tp.Series, **kwargs) -> None:
        checks.assert_instance_of(obj, pd.Series)
        BaseAccessor.__init__(self, obj, **kwargs)

    @class_or_instancemethod
    def is_series(cls_or_self) -> bool:
        return True

    @class_or_instancemethod
    def is_frame(cls_or_self) -> bool:
        return False
```

## class BaseDFAccessor(BaseAccessor)

```python
class BaseDFAccessor(BaseAccessor):
    def __init__(self, obj: tp.Frame, **kwargs) -> None:
        checks.assert_instance_of(obj, pd.DataFrame)
        BaseAccessor.__init__(self, obj, **kwargs)

    @class_or_instancemethod
    def is_series(cls_or_self) -> bool:
        return False

    @class_or_instancemethod
    def is_frame(cls_or_self) -> bool:
        return True
```